# Seminario de titulo

## Crear soluciones (no ejecutar si no es necesario)

### Crear Instancias

In [1]:
from clp.get_instance import get_uni

get_uni("instances",10,500)

### Crear soluciones solver

In [ ]:
import sys
sys.path.append("c:/Users/franc/Escritorio/Trabajo-final-de-titulo/src")

from solver.get_solver_solutions import generarSolucionesSolver

generarSolucionesSolver(500)

Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 0 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 1 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 2 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 3 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 4 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 5 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 6 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 7 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/s

## Reconstruir Bloques

In [1]:
import clp.base
import numpy as np
import math
import clp.box_plotter
import os

from clp.base import Space, BlockList, Block, Aabb
from clp.base import Block, Boxtype, Itemdict

import random

def get_location_block(block,space):
  x,y,z = space.corner_point
  if x == space.xmax: x -= block.l
  if y == space.ymax: y -= block.w
  if z == space.zmax: z -= block.h

  return x,y,z


def transformar_a_bloques(bloques,cajas_por_bloques,lwh2item):
    listaGeneral = []
    bloquesDict = dict()  #diccionario de bloques (n) = bloque[1] -> BlockType(bloque)



    for i in range(len(bloques)):

        bloque = bloques[i].split()
        lwhBloque = [int(valor) for valor in bloque[2].strip('()').split(',')]

        #hacer bl_items para cada bloque -> Diccionario de cajas para cada bloque
        bl_items = Itemdict()

        ##print(cajas_por_bloques[i]) # ['-- box: (49,25,21) n: 12', '-- box: (85,84,72) n: 2', '-- box: (48,36,31) n: 7']
        for j in cajas_por_bloques[i]:
            ##print(j) #-- box: (49,25,21) n: 12
            caja = j.split()
            lwhCaja = [int(valor) for valor in caja[2].strip('()').split(',')]
            n = int(caja[4])
            bl_items[lwh2item[(lwhCaja[0],lwhCaja[1],lwhCaja[2])]] = n


        newBlock = Block(l=lwhBloque[0],w = lwhBloque[1],h = lwhBloque[2],items = bl_items)
        #agregamos el bloque a la lista de bloques validos
        listaGeneral.append(newBlock)
        #agregamos el bloque al diccionario de bloques  [objeto bloque + id]
        #block: 2 (49,21,25)
        blockId = int(bloque[1])
        bloquesDict[blockId] = [i,newBlock] #el id del bloque es la clave, el valor es una lista con la posicion en la lista general y el objeto bloque

    return listaGeneral, bloquesDict


    ## obtener cajas desde archivo de texto
def obtener_registros_de_tipo_block(archivo):
    try:
        #lista
        bloques = []
        ##lista de listas
        cajas = []
        #abrir archivo
        with open(archivo, 'r') as f:
            #leer cada linea del archivo
            for line in f:
                #verificar si esta linea tiene la palabra "n_blocks:"
                if 'n_blocks:' in line:
                    #almacenamos el numero de cajas que se van a generar
                    n_blocks = int(line.split(':')[1])
                #verificar si esta linea tiene la palabra reservada "-- box:"
                if '-- box:' in line:
                    #asignar esa caja a la misma ubicacion que el bloque
                    cajas[len(bloques) - 1].append(line.strip())

                #verificar si esta linea tiene la palabra reservada "block:"
                if 'block:' in line:
                    #guardamos el bloque en bloques
                    bloques.append(line.strip())
                    #creamos una nueva lista de cajas
                    cajas.append([])
            #los primeros 10000 elementos de bloques son de tipo bloque, los demas son acciones
            f.close()
            return bloques[:n_blocks],cajas,bloques[n_blocks:]
    except FileNotFoundError:
        return f"El archivo '{archivo}' no se encontró."



# generar Space
def genSpace(L,W,H):

    Space.filling = "free"
    Space.vertical_stability = False

    cont = Block(l=L,w=W,h=H)

    space = cont.free_space.closest_space() # espacio libre donde se va a colocar el bloque
    return space,cont


## filtrar los elementos validos para que solo tomemos los que son validos para un space x


## Falta comprobar si funciona -> (recive la lista de bloques y la lista de items que quedan)
def remove_unconstructable(blocks, items):
        #cannot remove while iterating
    to_remove = list()
    for block in blocks:
        if not block.is_constructible(items):
                to_remove.append(block)
    for block in to_remove:
        blocks.remove(block)


    return blocks

def obtenerAcciones(acciones):
    seleccion = []
    acciones_candidatas = []
    

    for accion in acciones:
        '''selected block:82126 space:(0,0,0)
            action block:78058 eval: 7138616 0.035973469 0.78399454 0.045454545 1401104.8 
            action block:22894 eval: 6283256 0.0088177917 0.76397793 0.071428571 1251514.5 
            action block:22661 eval: 5834452 0.0088177917 0.75365579 0.076923077 1117010 
            action block:80927 eval: 5813328 0.0088177917 0.75365579 0.0625 1067693.2 
            action block:22878 eval: 5385648 0.0088177917 0.74238952 0.083333333 986466.95 
            action block:82126 eval: 6161392 0.012875536 0.61978122 0.055555556 503442.55 '''

        if "selected" in accion:    
            #Selected block
            x,y,z = accion.split()[2].split(":")[1].strip("()").split(",")
            selected_block = int(accion.split()[1].split(":")[1])
            seleccion.append([selected_block,int(x),int(y),int(z)])
            heuristic_eval = dict()
            acciones_candidatas.append([])

        if "action" in accion:
            block_id = int(accion.split()[1].split(":")[1])
            eval_block = accion.split()[3:]
            eval_block = [float(x) for x in eval_block] #if x.replace('.','',1).isdigit()
            heuristic_eval[block_id] = eval_block
            acciones_candidatas[len(seleccion) - 1] = heuristic_eval


        
    return seleccion, acciones_candidatas


def add_block(self, block, x, y, z):
    """
    Coloca un bloque en la posición especificada, si hay espacio disponible.
    Args:
        block (Block): El bloque que se desea colocar.
        x, y, z (int): Coordenadas de la esquina inferior izquierda del bloque.
    """
    # Crear el Aabb del bloque basado en las dimensiones y la posición.
    aabb = Aabb(x, x + block.l, y, y + block.w, z, z + block.h)

    # Verificar si el espacio está libre.
    """if not self.free_space.is_free(aabb):
        raise Exception(f"No hay suficiente espacio libre para colocar el bloque en la posición ({x}, {y}, {z})")"""

    # Colocar el bloque y recortar el espacio libre.
    self.aabbs.append(aabb)
    self.occupied_volume += block.occupied_volume
    self.weight += block.weight
    self.items += block.items
    self.free_space.crop(aabb, block)
    #print(f"Bloque colocado en posición ({x}, {y}, {z})")


Block.add_block = add_block


def funcionActualizadora(space, block, cont, items, blocks, bloquesDict, accion):

    block_loc = get_location_block(block, space) #-> guardar para generar el vector

    cont.add_block(block, accion[1],accion[2],accion[3])  # añadir el bloque al contenedor
    items -= block.items
    blocks = remove_unconstructable(blocks,items) #-> eliminar bloques que no son validos
    cont.free_space.filter(items)
    space = cont.free_space.closest_space()

    return space, block, cont, items, blocks, block_loc




def get_X(space, valid_blocks, bloques_colocados, cont, cant_cajas):
    #almacenamos las dimensiones del contenedor
    lwhSpace = [space.xmax, space.ymax, space.zmax]
    #next_loc = get_location_block(block, space)

    X = []
    #agregamos el space a la lista
    '''space
    - Dimensiones del bloque
    - posicion espacial del bloque
    - cantidad de cajas del bloque
    - si se encuentra colocado
    - si es contenedor'''
    space = [space.xmax,space.ymax,space.zmax,0,0,0,0,0,1] #las dimensiones del space estan normalizadas

    X.append(space)

    #para cada bloque colocado
    for element in bloques_colocados:
        block = element[1]
        block_loc = element[2]
        block_elements = block.items

        #normalizamos las dimensiones
        norm_dims = [block.l , block.w, block.h]

        #normalizamos la ubicacion
        norm_loc = [block_loc[0], block_loc[1], block_loc[2]]

        norm_elements = sum(block_elements.values())

        norm_block = norm_dims + norm_loc + [norm_elements, 1, 0]

        X.append(norm_block)


    #para cada bloque creamos un elemento => [dim normalizadas, ubicacion normalizada (si aun no esta colocado se deja como 0,0,0), si se encuentra colocado (1,0), si correponde al space (1,0), ]
    for block in valid_blocks:
        #normalizamos las dimensiones
        norm_dims = [block.l, block.w , block.h]



        #estos bloques no estan colocados
        norm_loc = [0, 0, 0]

        norm_elements = sum(block.items.values())


        norm_block = norm_dims + norm_loc + [norm_elements, 0, 0]

        X.append(norm_block)



    return X


def genVectorBinario(accion, bloquesDict, valid_blocks, cantidadElementos, cantidadElementosColocados):
    Y = np.zeros(cantidadElementos)

    blockId = accion[0]
    block = bloquesDict.get(blockId, None)

    if block is None:
        raise ValueError(f"El bloque con ID {blockId} no se encontró en bloquesDict.")

    try:
        posicionSinActualizar = valid_blocks.index(block)
    except ValueError:
        raise ValueError(f"El bloque con ID {blockId} no está en valid_blocks.")

    # +1 por el espacio contenedor, +cantidad de colocados
    posicion = posicionSinActualizar + 1 + cantidadElementosColocados
    if posicion >= cantidadElementos:
        raise IndexError(f"Posición {posicion} fuera del rango del vector Y de tamaño {cantidadElementos}.")

    Y[posicion] = 1
    return Y, posicionSinActualizar


def load_instance(filename = "instancia.txt", type="BF", id_instance=0):
    

    with open(filename, "r") as file:
        #discard first line
        file.readline()

        for j in range(1000):
            file.readline()
            L, W, H = [int(x) for x in file.readline().split()]

            #read second line
            n = int(file.readline())
            #read n lines
            items = Itemdict()
            lwh2item = dict()
            for i in range(n):
                id, l, rotx, w, roty, h, rotz, n = [int(x) for x in file.readline().split()]
                item = Boxtype(id, l, w, h, rotx, roty, rot_h=True, weight=1)
                items[item]=n
                lwh2item[(l,w,h)] = item

            if j == id_instance: return  items, L, W, H, lwh2item




def getSample(x_accion, y_accion, posicion, cantidad_bloques_colocados, tamanioMuestra=100):
    # Posición absoluta del bloque correcto dentro de x_accion
    posicion_elemento = posicion + 1 + cantidad_bloques_colocados
    block = x_accion[posicion_elemento]

    # 1. Eliminar el bloque seleccionado de la lista auxiliar
    x_aux = x_accion[:posicion_elemento] + x_accion[posicion_elemento + 1:]

    # 2. Eliminar el espacio y los bloques ya colocados
    x_aux = x_aux[1 + cantidad_bloques_colocados:]

    # 3. Seleccionar aleatoriamente hasta (tamanioMuestra - 1) elementos
    num_samples = min(tamanioMuestra - 1, len(x_aux))
    muestra = random.sample(x_aux, num_samples)

    # 4. Padding si hay menos elementos que tamanioMuestra - 1
    feature_len = len(block)
    while len(muestra) < tamanioMuestra - 1:
        muestra.append([0] * feature_len)

    # 5. Armar muestra final: espacio + colocados + muestra + bloque correcto
    x_sample = x_accion[:1 + cantidad_bloques_colocados] + muestra
    x_sample.append(block)

    # 6. Etiqueta: 1 solo al final
    y_sample = np.zeros(len(x_sample))
    y_sample[-1] = 1

    return np.array(x_sample), np.array(y_sample)

In [2]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences  # ← correcto


def generarX_y(id):
    #abrimos el archivo que contiene el resultado del problema
    #tests\resultados_solver\output1.txt
    archivo = r"tests\resultados_solver\output"+str(id+1)+".txt"

    #obtenemos los datos importantes del archivo
    bloques,cajas,acciones = obtener_registros_de_tipo_block(archivo)  ##(acciones)

    #cargamos la instancia original
    items, L, W, H, lwh2item = load_instance(filename = r"tests\instances\instances.txt", type="BF", id_instance=id)

    acciones, acciones_candidatas = obtenerAcciones(acciones)

    #obtenemos la cantidad de cajas
    cant_cajas = sum(items.values())

    #definimos las variables a utilizar
    valid_blocks, bloquesDict = transformar_a_bloques(bloques,cajas,lwh2item)
    space,cont = genSpace(L,W,H)
    bloques_colocados = []
    X = []
    Y = []

    for accion in acciones:
        #print(space)
        if(space == None): break
        x_accion = get_X(space, valid_blocks, bloques_colocados, cont, cant_cajas)
        y_accion, posicion = genVectorBinario(accion, bloquesDict, valid_blocks, len(x_accion), len(bloques_colocados))

        x_sample,y_sample = getSample(x_accion, y_accion, posicion, len(bloques_colocados), 100)
        space, bloque_actual, cont, items, valid_blocks, block_loc = funcionActualizadora (space, valid_blocks[posicion], cont, items, valid_blocks, bloquesDict,accion)


        id = accion[0]
        bloques_colocados.append([id, bloque_actual, block_loc])

        X.append(x_sample)
        Y.append(y_sample)



    #mostrar la solucion
    #box_dims = []
    #for aabb in cont.aabbs:
        #box_dims.append([aabb.xmin,aabb.ymin,aabb.zmin,aabb.xmax,aabb.ymax,aabb.zmax])

    #box_plotter.plot_container([L,W,H], box_dims)
    return X,Y

def generate_data(runs = 100):
    X = []
    Y = []

    Max_blocks = 135

    #generar los datos
    for i in range(runs):
        print(i)
        x,y = generarX_y(i)

        X += x
        Y += y


    X_padded = pad_sequences(X, padding='post',dtype=np.float64)
    Y_padded = pad_sequences(Y, padding='post',dtype=np.float64)

    

    # 3. Normalización Min-Max por característica
    # Flatten temporalmente para normalizar
    max_blocks = X_padded.shape[2]

    X_reshaped = X_padded.reshape(-1, 9)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_reshaped)

    # Restaurar forma original
    X_scaled = X_scaled.reshape(X_padded.shape)


    return np.array(X_scaled),np.array(Y_padded)


X, y = generate_data(1)
print(X)

np.savez_compressed("datos_muestras.npz", X=X, Y=y)




KeyboardInterrupt: 

In [3]:
from clp.base import Block
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences 

def data_gen_encoder(muestra = 500):
    data_encoder = []
    for i in range(muestra):
        #carga de datos
        archivo = r"tests\resultados_solver\output"+str(i+1)+".txt"

        #obtenemos los datos importantes del archivo
        bloques,cajas,acciones = obtener_registros_de_tipo_block(archivo)  ##(acciones)

        # Encoder
        #bloques disponibles (10000 , 5), cajas disponibles (10, 4)(Dict), Contenedor (1, 3) + (restricciones)

        items, L, W, H, lwh2item = load_instance(filename = r"tests\instances\instances.txt", type="BF", id_instance=i)
        cajas_disponibles = []
        contenedor_inicial = [L,W,H] # dimensiones del contenedor
        for caja in items:
            cajas_disponibles.append([caja.l, caja.w, caja.h, items[caja]]) # cajas disponibles

        valid_blocks, bloquesDict = transformar_a_bloques(bloques,cajas,lwh2item)
        bloques_disponibles = []
        for block in valid_blocks:
            bloques_disponibles.append([block.l, block.w, block.h, block.volume, block.occupied_volume])

            

    



        





data_gen_encoder(1)

In [4]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def normalizar_minmax_global(lista_de_pasos, clip_quantiles=(0.005, 0.995)):
    """
    lista_de_pasos: [
        [[id, h1, h2, h3, h4, h5], ...],   # paso 0
        [[id, h1, h2, h3, h4, h5], ...],   # paso 1
        ...
    ]
    Devuelve:
      - lista_norm: misma estructura, con h1..h5 escalados globalmente a [0,1]
      - scaler: MinMaxScaler ajustado globalmente
      - stats: dict con medianas y límites de clip por columna (para aplicar coherente en nuevos datos)
    """
    # 1) Reunir todo a nivel global (solo heurísticos)
    all_rows = []
    for paso in lista_de_pasos:
        for fila in paso:
            all_rows.append(fila[1:])  # [h1..h5]
    if not all_rows:
        return [[] for _ in lista_de_pasos], MinMaxScaler(), {"median": None, "clip_low": None, "clip_high": None}

    X_all = np.asarray(all_rows, dtype=float)  # (N, 5)

    # 2) Reemplazar ±inf por NaN (MinMaxScaler permite NaN, pero no ±inf)
    X_all[~np.isfinite(X_all)] = np.nan

    # 3) Imputar por mediana (ignora NaN). Si toda la columna es NaN, usa 0.
    median = np.nanmedian(X_all, axis=0)
    median = np.where(np.isfinite(median), median, 0.0)
    # aplicar imputación
    inds_nan = np.isnan(X_all)
    if inds_nan.any():
        X_all[inds_nan] = np.take(median, np.where(inds_nan)[1])

    # 4) (Opcional) Clip por cuantiles para reducir outliers extremos
    if clip_quantiles is not None:
        qlow, qhigh = clip_quantiles
        clip_low  = np.quantile(X_all, qlow, axis=0)
        clip_high = np.quantile(X_all, qhigh, axis=0)
        X_all = np.clip(X_all, clip_low, clip_high)
    else:
        clip_low  = None
        clip_high = None

    # 5) Ajustar scaler global
    scaler = MinMaxScaler()
    scaler.fit(X_all)

    # 6) Transformar cada paso aplicando las MISMAS reglas (inf→NaN, imputar, clip, escalar)
    lista_norm = []
    for paso in lista_de_pasos:
        if not paso:
            lista_norm.append([])
            continue
        ids  = np.array([int(f[0]) for f in paso], dtype=int)
        vals = np.array([f[1:] for f in paso], dtype=float)

        # limpiar
        vals[~np.isfinite(vals)] = np.nan
        # imputar con la mediana global
        mask = np.isnan(vals)
        if mask.any():
            vals[mask] = np.take(median, np.where(mask)[1])
        # clip con límites globales
        if clip_low is not None:
            vals = np.clip(vals, clip_low, clip_high)
        # escalar
        vals_norm = scaler.transform(vals)

        paso_norm = np.column_stack([ids, vals_norm]).tolist()
        lista_norm.append(paso_norm)

    stats = {"median": median, "clip_low": clip_low, "clip_high": clip_high}
    return lista_norm, scaler, stats

In [27]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def data_gen_encoder(muestra=500):
    data_encoder = []
    data_decoder = []
    Y = []
    for i in range(muestra):
        # carga de datos
        archivo = r"tests\resultados_solver\output" + str(i+1) + ".txt"

        # obtenemos los datos importantes del archivo
        bloques, cajas, acciones = obtener_registros_de_tipo_block(archivo)

        # Encoder
        items, L, W, H, lwh2item = load_instance(
            filename=r"tests\instances\instances.txt",
            type="BF",
            id_instance=i
        )
        
        contenedor_inicial = np.array([1, 1, 1], dtype=float)  # dimensiones del contenedor
        
        # -------------------------------
        # Normalización de CAJAS
        # -------------------------------
        cajas_raw = []
        for caja in items:
            cajas_raw.append([caja.l, caja.w, caja.h, items[caja]])  # (l, w, h, cantidad)
        
        cajas_raw = np.array(cajas_raw, dtype=float)
        
        # Normalizamos dimensiones respecto al contenedor
        cajas_raw[:, 0] /= L
        cajas_raw[:, 1] /= W
        cajas_raw[:, 2] /= H
        
        # Normalizamos solo la cantidad (columna 3)
        scaler_cajas = MinMaxScaler()
        cantidades = cajas_raw[:, 3].reshape(-1, 1)  # convertir a 2D
        cajas_raw[:, 3] = scaler_cajas.fit_transform(cantidades).flatten()

        cajas_norm = cajas_raw  # ya está normalizado

        
        # -------------------------------
        # Normalización de BLOQUES
        # -------------------------------
        valid_blocks, bloquesDict = transformar_a_bloques(bloques, cajas, lwh2item)
        
        bloques_raw = []
        for block in valid_blocks:
            bloques_raw.append([block.l, block.w, block.h, block.volume, block.occupied_volume])
        
        bloques_raw = np.array(bloques_raw, dtype=float)

        if len(bloques_raw) > 0:
            # Normalizamos dimensiones con respecto al contenedor
            bloques_raw[:, 0] /= L
            bloques_raw[:, 1] /= W
            bloques_raw[:, 2] /= H

            # volúmenes normalizados respecto al volumen del contenedor
            volumen_contenedor = L * W * H
            bloques_raw[:, 3] /= volumen_contenedor
            bloques_raw[:, 4] /= volumen_contenedor

            bloques_norm = bloques_raw  # ya está normalizado

        else:
            bloques_norm = np.empty((0, 5))

        # -------------------------------
        # Guardamos todo en data_encoder
        # -------------------------------
        data_encoder.append(
            [cajas_norm,
            bloques_norm,
            contenedor_inicial]
        )

        # -------------------------------
        # Generacion data_decoder (paso a paso)
        # -------------------------------
        seleccion, acciones_candidatas = obtenerAcciones(acciones)

        acciones_candidatas_list = []
        espacio_libre = []
        for j,paso in enumerate(acciones_candidatas):
            acciones_candidatas_list.append([])
            for key,value in paso.items():
                acciones_candidatas_list[j].append([bloquesDict[key][0]] + value)

        acciones_candidatas_list, scaler_acciones_candidatas, stats = normalizar_minmax_global(acciones_candidatas_list)

        for paso in seleccion:
            espacio_libre.append([paso[1]/L, paso[2]/W, paso[3]/H])
        
        acciones_candidatas_list = pad_sequences(acciones_candidatas_list, padding='post', dtype='float32')
            
        # -------------------------------
        # Guardamos todo en data_decoder
        # -------------------------------
        data_decoder.append(
            [acciones_candidatas_list,
            espacio_libre]
        )

        # -------------------------------
        # Generacion de salida (paso a paso)
        # -------------------------------
        pasos_list = []
        for paso in seleccion:
            y = np.zeros(10000)
            y[bloquesDict[paso[0]][0]] = 1
            pasos_list.append(y)

        Y.append(pasos_list)

    
    return data_encoder, data_decoder, Y


Data_encoder, data_decoder, Y = data_gen_encoder(500)

In [31]:
from keras.preprocessing.sequence import pad_sequences

Y_padded = pad_sequences(Y, padding='post', dtype='float32')


data_decoder_padded = pad_sequences(data_decoder, padding='post', dtype='float32')


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 21) + inhomogeneous part.

In [29]:
np.savez_compressed("datos_encoder.npz", data=Data_encoder)
np.savez_compressed("datos_decoder.npz", data=data_decoder)
np.savez_compressed("datos_salida.npz", data=Y_padded)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (500, 3) + inhomogeneous part.

In [21]:
print(Y_padded.shape)

(500, 35, 10000)


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences

# -------------------------------------------------
# Parámetros globales (ajusta si cambian)
# -------------------------------------------------
T_max = 35
N_max = 6
N_blocks = 10000
F_block = 5   # [l, w, h, vol_norm, occ_vol_norm]

def generar_datasets_opcion1(muestra=500):
    # Contenedores temporales
    boxes_list, box_m_list, cont_list = [], [], []
    space_list, ids_list, heur_list, cmask_list = [], [], [], []
    Y_list, has_step_list = [], []

    # Tabla global de bloques (llenamos según aparezcan índices)
    blocks_table = np.zeros((N_blocks, F_block), dtype=np.float32)
    seen_idx = set()

    # Dimensionalidad heurística (se detecta una vez)
    F_h_global = None

    for i in range(muestra):
        # ---------------------
        # 1) Cargar y parsear
        # ---------------------
        archivo = rf"tests\resultados_solver\output{i+1}.txt"
        bloques, cajas, acciones = obtener_registros_de_tipo_block(archivo)

        items, L, W, H, lwh2item = load_instance(
            filename=r"tests\instances\instances.txt",
            type="BF",
            id_instance=i,
        )

        # -------------------------------
        # 2) Encoder: Cajas normalizadas
        # -------------------------------
        contenedor_inicial = np.array([1.0, 1.0, 1.0], dtype=np.float32)
        cont_list.append(contenedor_inicial)

        cajas_raw = []
        for caja in items:
            cajas_raw.append([caja.l, caja.w, caja.h, items[caja]])  # (l,w,h,cantidad)

        cajas_raw = np.array(cajas_raw, dtype=np.float32) if len(cajas_raw) else np.zeros((0,4), np.float32)

        if cajas_raw.size > 0:
            # dimensiones relativas al contenedor
            cajas_raw[:, 0] /= L
            cajas_raw[:, 1] /= W
            cajas_raw[:, 2] /= H
            # cantidad normalizada por MinMax
            scaler_cajas = MinMaxScaler()
            cantidades = cajas_raw[:, 3].reshape(-1, 1)
            cajas_raw[:, 3] = scaler_cajas.fit_transform(cantidades).flatten()

        boxes_list.append(cajas_raw.astype(np.float32))
        box_m_list.append(np.ones((cajas_raw.shape[0],), dtype=np.float32))

        # ------------------------------------------------
        # 3) Blocks → normalizar y volcar en tabla global
        # ------------------------------------------------
        valid_blocks, bloquesDict = transformar_a_bloques(bloques, cajas, lwh2item)
        volumen_contenedor = float(L * W * H)

        def _norm_block_features(block_obj):
            # l,w,h normalizados por el contenedor
            l = float(block_obj.l) / L
            w = float(block_obj.w) / W
            h = float(block_obj.h) / H
            # volumenes (con fallback por si el objeto no trae los atributos)
            vol_abs = float(getattr(block_obj, "volume", block_obj.l * block_obj.w * block_obj.h))
            occ_abs = float(getattr(block_obj, "occupied_volume", vol_abs))
            v  = vol_abs / volumen_contenedor
            vo = occ_abs / volumen_contenedor
            return l, w, h, v, vo

        # Rellena la tabla global usando el índice entero que usas en todo el pipeline:
        #   idx_en_listaGeneral = bloquesDict[blockId][0]
        #   block_obj           = bloquesDict[blockId][1]
        for blockId, (idx_in_list, block_obj) in bloquesDict.items():
            idx = int(idx_in_list)  # este es el índice que usas en ids/Y
            if 0 <= idx < N_blocks:
                l, w, h, v, vo = _norm_block_features(block_obj)
                blocks_table[idx] = [l, w, h, v, vo]
                seen_idx.add(idx)

        # ------------------------------------------------
        # 4) Decoder: candidatos → ids/heur/cmask + space
        # ------------------------------------------------
        seleccion, acciones_candidatas = obtenerAcciones(acciones)
        # acciones_candidatas es lista de dicts: [{block_id: feat_vec, ...}, ...]

        # espacio libre por paso
        space_steps = []
        for paso in seleccion:
            # paso = (block_id, x, y, z, ...) → ejemplo tuyo
            space_steps.append([paso[1] / L, paso[2] / W, paso[3] / H])
        space_steps = np.array(space_steps, dtype=np.float32) if len(space_steps) else np.zeros((0,3), np.float32)

        # separar ids / heur / cmask por paso
        ids_steps, heur_steps, cmask_steps = [], [], []
        local_F_h = None

        for paso_dict in acciones_candidatas:
            cand_ids, cand_feats = [], []
            for key, value in paso_dict.items():
                # key: block reference → índice global
                idx = int(bloquesDict[key][0])
                cand_ids.append(idx)
                cand_feats.append(value)

            n_cand = len(cand_ids)
            if n_cand > 0:
                # fijar F_h global si no está
                if F_h_global is None:
                    F_h_global = len(cand_feats[0])
                local_F_h = len(cand_feats[0])

            # pad a N_max
            ids_p = np.zeros((N_max,), dtype=np.int32)
            if n_cand > 0:
                ids_p[:min(n_cand, N_max)] = np.array(cand_ids[:N_max], dtype=np.int32)
            ids_steps.append(ids_p)

            cm = np.zeros((N_max,), dtype=np.float32)
            cm[:min(n_cand, N_max)] = 1.0
            cmask_steps.append(cm)

            if n_cand > 0:
                feats = np.array(cand_feats[:N_max], dtype=np.float32)  # [min(N_max), F_h]
                # pad N_max × F_h_global (temporal; ajustamos F_h más abajo)
                heur_steps.append(feats)
            else:
                heur_steps.append(np.zeros((0, 0), dtype=np.float32))

        # pad en tiempo a T_max
        def pad_time(arr, target_shape, pad_value=0.0, dtype=np.float32):
            out = np.full(target_shape, pad_value, dtype=dtype)
            if arr.size == 0:
                return out
            slices = tuple(slice(0, min(a, t)) for a, t in zip(arr.shape, target_shape))
            out[slices] = arr[slices]
            return out

        # space [T_max, 3]
        space_list.append(pad_time(space_steps, (T_max, 3), 0.0, np.float32))

        # ids [T_max, N_max] (int32)
        ids_arr = np.stack(ids_steps, axis=0) if len(ids_steps) else np.zeros((0, N_max), np.int32)
        ids_list.append(pad_time(ids_arr, (T_max, N_max), 0, np.int32))

        # cmask [T_max, N_max]
        cmask_arr = np.stack(cmask_steps, axis=0) if len(cmask_steps) else np.zeros((0, N_max), np.float32)
        cmask_list.append(pad_time(cmask_arr, (T_max, N_max), 0.0, np.float32))

        # heur: primero stack sin forzar F_h, luego ajustaremos a F_h_global
        if len(heur_steps) and heur_steps[0].size > 0:
            # rellena cada paso a N_max x local_F_h (ya viene así implícitamente)
            # lo paddeamos en tiempo luego
            # detectaremos F_h_global a nivel dataset (primera vez que haya candidatos)
            pass
        # guardamos temporalmente objetos Python; al final homogenizamos

        heur_list.append(heur_steps)  # guardo la lista "raw" por muestra

        # ---------------------------------
        # 5) Targets: Y one-hot + has_step
        # ---------------------------------
        Y_steps = []
        for paso in seleccion:
            y = np.zeros((N_blocks,), dtype=np.float32)
            y[bloquesDict[paso[0]][0]] = 1.0
            Y_steps.append(y)

        Y_i = np.stack(Y_steps, axis=0) if len(Y_steps) else np.zeros((0, N_blocks), np.float32)
        Y_list.append(pad_time(Y_i, (T_max, N_blocks), 0.0, np.float32))

        has = np.zeros((T_max,), dtype=np.float32)
        has[:len(Y_steps)] = 1.0
        has_step_list.append(has)

    # -----------------------------
    # Homogeneizar dimensiones
    # -----------------------------
    # 1) Boxes → pad a N_box_max
    N_box_max = max(b.shape[0] for b in boxes_list) if boxes_list else 0
    boxes = np.zeros((muestra, N_box_max, 4), dtype=np.float32)
    box_m = np.zeros((muestra, N_box_max), dtype=np.float32)
    for i, (b, m) in enumerate(zip(boxes_list, box_m_list)):
        if b.shape[0] > 0:
            boxes[i, :b.shape[0], :] = b
            box_m[i, :m.shape[0]] = m

    cont = np.stack(cont_list, axis=0).astype(np.float32)  # [B, 3]

    # 2) Decoder: ids, cmask, space ya están paddeados
    space = np.stack(space_list, axis=0).astype(np.float32)                  # [B, T_max, 3]
    ids = np.stack(ids_list, axis=0).astype(np.int32)                        # [B, T_max, N_max]
    cmask = np.stack(cmask_list, axis=0).astype(np.float32)                  # [B, T_max, N_max]

    # Heur: homogenizar a F_h_global y pad temporal
    if F_h_global is None:
        F_h_global = 0  # no hubo candidatos en todo el dataset
    heur_np = np.zeros((muestra, T_max, N_max, F_h_global), dtype=np.float32)
    for i, heur_steps in enumerate(heur_list):
        T_i = min(T_max, len(heur_steps))
        for t in range(T_i):
            feats = heur_steps[t]
            if feats.size == 0:
                continue
            # feats shape: [<=N_max, F_h_local]
            n_here = min(N_max, feats.shape[0])
            f_here = min(F_h_global, feats.shape[1])
            heur_np[i, t, :n_here, :f_here] = feats[:n_here, :f_here]

    # 3) Targets
    Y_onehot = np.stack(Y_list, axis=0).astype(np.float32)                   # [B, T_max, N_blocks]
    has_step = np.stack(has_step_list, axis=0).astype(np.float32)            # [B, T_max]

    # 4) Placeholders para compatibilidad (no se usarán en el modelo con gather)
    blocks_ph = np.zeros((muestra, 1, F_block), dtype=np.float32)
    blk_m_ph = np.zeros((muestra, 1), dtype=np.float32)

    # 5) Guardar a disco

    # 1) Tabla global en fp16 (para mmap desde el modelo)
    np.save("blocks_table_fp16.npy", blocks_table.astype(np.float16))

    # 2) Encoder: ¡sin blocks/blk_m!
    np.savez_compressed(
        "datos_encoder.npz",
        boxes=boxes,     # [B, N_box_max, 4]
        cont=cont,       # [B, 3]
        box_m=box_m      # [B, N_box_max]
    )

    # 3) Decoder (igual que antes)
    np.savez_compressed(
        "datos_decoder.npz",
        space=space,     # [B, T_max, 3]
        ids=ids,         # [B, T_max, N_max]
        heur=heur_np,    # [B, T_max, N_max, F_h]
        cmask=cmask,     # [B, T_max, N_max]
        T_max=np.array(T_max, dtype=np.int32),
        N_max=np.array(N_max, dtype=np.int32),
        F_h=np.array(F_h_global, dtype=np.int32)
    )

    # 4) Targets
    np.savez_compressed("datos_salida.npz", Y=Y_onehot, has_step=has_step)

    print(f"OK: guardado blocks_table_fp16.npy, datos_encoder.npz, datos_decoder.npz, datos_salida.npz")
    print(f"Shapes → boxes {boxes.shape}, cont {cont.shape}, box_m {box_m.shape}")
    print(f"Decoder → space {space.shape}, ids {ids.shape}, heur {heur_np.shape}, cmask {cmask.shape}")
    print(f"Targets → Y {Y_onehot.shape}, has_step {has_step.shape}")
    print(f"Blocks vistos: {len(seen_idx)} / {N_blocks}")

# Lanza la generación
generar_datasets_opcion1(muestra=500)

OK: guardado blocks_table_fp16.npy, datos_encoder.npz, datos_decoder.npz, datos_salida.npz
Shapes → boxes (500, 10, 4), cont (500, 3), box_m (500, 10)
Decoder → space (500, 35, 3), ids (500, 35, 6), heur (500, 35, 6, 5), cmask (500, 35, 6)
Targets → Y (500, 35, 10000), has_step (500, 35)
Blocks vistos: 0 / 10000
